In [1]:

import numpy as np
from rich.progress import track

class Tanh:
    def activate(self, x):
        return (np.exp(x) - np.exp(-x)) / (np.exp(x) + np.exp(-x))
    
    def derivative(self, x):
        return 1 - self.activate(x) ** 2


class Layer:
    def __init__(self, len_inputs, neurons, function, last=False):
        shape = neurons, len_inputs + 1
        self.weights = np.random.uniform(-0.5, 0.5, size=shape)
        self.f = function
        self.last = last
        self.idx = None
        self.neurons = neurons
        self.len_inputs = len_inputs
    
    def forward(self, layer_input):
        self.input = layer_input
        self.net = self.input.dot(self.weights.T)
        self.output = self.f.activate(self.net)
        return self.output
    
    def backward(self, target, alpha, previous_delta=None, previous_weigth=None):
        if self.last:
            self.delta = (target - self.output) * self.f.derivative(self.net)
        else:
            self.delta = (np.delete(previous_delta.dot(previous_weigth).T, 0) * self.f.derivative(self.net))
        
        self.weights += np.array([self.delta]).T * np.array([self.input]) * alpha
        
        return self.delta, self.weights
        
        
        
        
    def __repr__(self):
        return f"({self.idx}º Layer, Neurons: {self.neurons}, Last: {self.last})"


class NeuralNetwork:
    def __init__(self, *layers: Layer):
        self.layers = list(layers)
        for idx, layer in enumerate(self.layers):
            layer.idx = idx + 1
        self.layers[-1].last = True
        self.len_inputs = self.layers[0].len_inputs
        
    def __repr__(self):
        return f"NeuralNetwork (Num_Layers: {len(self.layers)}, Len_Inputs: {self.len_inputs}, Layers: {self.layers})"
    
    @property
    def weights(self):
        resp = []
        for idx, layer in enumerate(self.layers):
            resp.append((idx+1, layer.weights))
        return resp
        
    def _forward(self, x_input):
        #input_layer = x_input
        input_layer = np.append(1, x_input)
        for layer in self.layers:
            out_layer = layer.forward(input_layer)
            input_layer = np.append(1, out_layer)
            
        return out_layer
    
    def _backward(self, y, alpha):
        for layer in reversed(self.layers):
            if layer.last:
                previous_delta, previous_weigth = layer.backward(y, alpha)
            else:
                previous_delta, previous_weigth = layer.backward(y, alpha, previous_delta, previous_weigth)
    
    def fit(self, x_train, y_train, epochs=2000, alpha=0.05):

        for epoch in track(range(epochs), description="Processing..."):
            outputs = []
            for x, y in zip(x_train, y_train):
                out = self._forward(x)
                self._backward(y, alpha)
                outputs.append(out)
                
            errors = np.array([sum(error) for error in (y_train - outputs) ** 2])
            self.mean_squared_error = sum(errors) / len(errors)
            
            if not epoch % 100:
                print(f"MSE: {self.mean_squared_error}")
                
                
    def predict(self, x):
        out = self._forward(x)
        return out



In [2]:
from rich.traceback import install; install();

In [3]:
x_train = [0, .1, .2, .3, .4, .5, .6, .7, .8, .9, 1.0]
y_train = [-.9602, -.5770, -.0729, .3771, .6405, .6600, .4609, .1336, -.2013, -.4344, -.5000 ]
x_train = np.array([[x, x] for x in x_train])
y_train = np.array([[y, y] for y in y_train])


rede = NeuralNetwork(
    Layer(len_inputs=2, neurons=4, function=Tanh()),
    Layer(len_inputs=4, neurons=2, function=Tanh()),
)
rede.fit(x_train, y_train)

Output()

MSE: 0.70335490709381

MSE: 0.5870951567302061

MSE: 0.6015914304873866

MSE: 0.5811064102858041

...

MSE: 0.0026829515979268142

MSE: 0.002656566387437313

MSE: 0.002631666902057951

MSE: 0.0026081400014337686

MSE: 0.002585887564408645

In [6]:
import polars as pl
import matplotlib.pyplot as plt

In [7]:
data = pl.read_csv("train.csv")
data.head()

label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,…,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
y_train = np.array(data.drop_in_place("label"))
y_train

array([1, 0, 1, ..., 7, 6, 9])

In [9]:
x_train = np.array([row for row in data.rows()]) / 255
len(x_train[0])

784

In [10]:
y_train[243]

def number_to_neurons(n):
    res = [-1] * 10
    res[n] = 1
    return res

y_train = np.array([number_to_neurons(y) for y in y_train])
y_train

array([[-1,  1, -1, ..., -1, -1, -1],
       [ 1, -1, -1, ..., -1, -1, -1],
       [-1,  1, -1, ..., -1, -1, -1],
       ...,
       [-1, -1, -1, ...,  1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1,  1]])

In [17]:
rede = NeuralNetwork(
    Layer(len_inputs=784, neurons=10, function=Tanh()),
    Layer(len_inputs=10, neurons=10, function=Tanh()),
)
rede.fit(x_train[:1000], y_train[:1000])

Output()

MSE: 3.0936188268200424

MSE: 0.2176520370408117

MSE: 0.20016953732195908

MSE: 0.20059306675314914

...

MSE: 0.1642665548375541

MSE: 0.16424013754302774

MSE: 0.1607925189132168

MSE: 0.1607474536646079

MSE: 0.16072981750381599

In [16]:
x_train[0:10]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [31]:
np.argmax(y_train[145])

4

In [32]:
np.argmax(rede.predict(x_train[145]))

4

In [48]:
total = 42000
points = 0
for idx in range(1000,total):
    correct = np.argmax(y_train[idx])
    predict = np.argmax(rede.predict(x_train[idx]))
    if correct == predict:
        points += 1


In [50]:
points/total * 100

79.31428571428572

In [57]:
def predizer_garrancho(garrancho):
    if garrancho is not None:
        x = garrancho.reshape(1, 784) / 255
        return np.argmax(rede.predict(x))




import gradio as gr

gr.Interface(fn=predizer_garrancho, 
             inputs="sketchpad",
             outputs="textbox",
             live=True).launch(share=True)

Running on local URL:  http://127.0.0.1:7866
Running on public URL: https://1ad72f454f7daffb2a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
